In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import os

In [ ]:
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True, warn_only=True)

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

torch.manual_seed(40)
torch.cuda.manual_seed_all(40)


In [ ]:
!wget https://www.dropbox.com/scl/fi/w22pt8h447b9ptgql67vo/dataset.tar.gz?rlkey=vajo7g4w8nl1q92ikv8qu75qu&dl=0

In [ ]:
!mv dataset.tar.gz?rlkey=vajo7g4w8nl1q92ikv8qu75qu dataset.tar.gz

In [ ]:
!tar -xzvf dataset.tar.gz

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'


In [ ]:
def read_image_tensor(image_folder,transform,num_images=None):
    if num_images==None:
        num_images = len(os.listdir(image_folder))
    images = []
    for i in range(num_images):
        img = torchvision.io.read_image(os.path.join(image_folder,f"{i}.jpg")).float()
        images.append(transform(img))
    return torch.stack(images).to(device)



In [ ]:
def get_labels(csv_file):
    # TODO: Return a torch tensor after reading the labels in csv_file. Convert to float().
    pass

In [ ]:
img_size = (256,256)
base_transform = transforms.Compose(
    [transforms.Resize(img_size)
    ]
)
train_X = read_image_tensor("dataset/train/",base_transform)/256
train_Y = get_labels("dataset/train.csv")
test_X = read_image_tensor("dataset/test/",base_transform)/256
test_Y = get_labels("dataset/test.csv")

In [ ]:
def train_model(model, train_loader, test_loader, num_epochs, loss_function, optimizer):
    # TODO: Make sure you read through these lines of code and understand all key lines.
    # For example: Why do you need to zero out the gradients using optimizer.zero_grad() in the for loop?
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs,labels = data
            optimizer.zero_grad()
            output = model(inputs)
            loss = loss_function(output,labels.view(output.shape))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        average_loss = total_loss/len(train_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                inputs, labels = data
                outputs = model(inputs)
                pred = (outputs > 0.5)*1
                correct += (pred==labels.view(pred.shape)).sum()
                total += labels.size(0)
            accur = 100*correct/total
            print(f"Test Accuracy after Epoch {epoch+1}: {accur:.2f}%")

    print("Training done.")


In [ ]:
# PART 1: TODO
# Write down the model description
# model = ...
# Relevant torch.nn classes you will need include nn.Sequential, nn.Conv2d, nn.MaxPool2d and so on.
model = #TODO

In [ ]:
model.to(device)

In [ ]:
train_dataset = TensorDataset(train_X, train_Y)
test_dataset = TensorDataset(test_X, test_Y)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
num_epochs = 30
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
train_model(model,train_loader,test_loader,num_epochs,loss_func,optimizer)

In [ ]:
def train_model_with_augment(model, augment_layer, train_loader, test_loader, num_epochs, loss_function, optimizer):


    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs,labels = data
            optimizer.zero_grad()
            output = model(augment_layer(inputs))
            loss = loss_function(output,labels.view(output.shape))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        average_loss = total_loss/len(train_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                inputs, labels = data
                outputs = model(inputs)
                pred = (outputs > 0.5)*1
                correct += (pred==labels.view(pred.shape)).sum()
                total += labels.size(0)
            accur = 100*correct/total
            print(f"Test Accuracy after Epoch {epoch+1}: {accur:.2f}%")

    print("Training done.")

In [ ]:
optimizer = optim.Adam(model.parameters())
loss_func = nn.BCELoss()
# PART 2: TODO
# Chain together transforms to create a set of allowed transformations within augment
# Available transformations are listed here: https://pytorch.org/vision/0.9/transforms.html
# Rotation, ColorJitter are popular transforms
from torchvision.transforms import v2
augment = # TODO: Define
train_model_with_augment(model, augment, train_loader, test_loader, num_epochs, loss_func, optimizer)